# ITPS Flight Test Analysis Tool - v2.5_alpha

In [ ]:
from FTAT_imports import *

# make display 100% of the screen
display(HTML("<style>.container { width:100% !important; }</style>"))
header_message = Label('Load Flight Data First')

In [ ]:
display(header_message)

In [ ]:
# Data File Selection


# set this variable to True to bypass file import and generate fake data
unit_test = False

raw_data = pd.DataFrame()
filetype = ''
file_status_label = widgets.Label('Choosing file phase:')
data_statistics_label1 = widgets.Label('')
data_statistics_label2 = widgets.Label('')
data_statistics_label3 = widgets.Label('')
f = FileChooser('/home/jovyan/work/data/')
####
#first load fake data to start
raw_data, filetype = load_data(True, f, None)
####
if not unit_test:
    # Create and display a FileChooser widget

    pass
else:
    raw_data, filetype = load_data(unit_test, f, None)

# Let's subsample to speed up the graphing process
desired_sub_sample_rate = 5 #Hz
sub_sample_rate, counter = get_sub_sample_rate(raw_data, desired_sub_sample_rate)
    
sub_sampled_data = raw_data[::sub_sample_rate]

time_slices_db = {} #this is the time slices data base, TP#, Description, Start, Stop (indexes)
selected_slice = [] #this is a global variable to store the currently selected slice that will be stored in the time_slices_db
map_groups = {} #ths is for grouping parameters in the map

######################### Slice Map and Definitions #############################################################################
for p in sub_sampled_data.columns:
    map_groups[p] = 'FTI'
slicemap = ParameterMap(sub_sampled_data, 'Parameter Map', map_groups)

######################### Slice Graph Definitions #############################################################################

current_plot_data = sub_sampled_data

slice_plot = LinePlotBrush(current_plot_data.index.values, current_plot_data.index.values)
detail_plot = LinePlot(current_plot_data.index.values, current_plot_data.index.values)
detail_plot_stats = widgets.HTML(value="Empty <b>Empty</b>", placeholder='Stats', description='Stats')
detail_plot_stats.value = 'Empty'

slicebox = sliceSelectDialog(current_plot_data)

######################### Parameter Selection and Detail Plot Updater #########################################


def detail_plot_update_brush(*args):
    lib_detail_plot_update_brush(*args, cpt=current_plot_data, selsl=selected_slice, detplt=detail_plot, slcplt=slice_plot, \
                                 slcbx=slicebox, dtstats=detail_plot_stats)

slice_plot.brushintsel.observe(detail_plot_update_brush, 'selected')


def selected_index_changed(change):
    '''
    This callback treats the parameter selection/deselection on the map
    Depending on the active tab, it will also callback the appropriate graph updater
    '''
    global current_plot_data, sub_sampled_data
    selected_tab = FTAT_tabs.selected_index
    if (selected_tab == 1 or selected_tab ==3):
        #this is for time slice or analysis tabs
        #define the current plot data per selected parameters
        if len(slicemap.map.selected) != 0:
            selected_parameters_plot_data = np.transpose(sub_sampled_data[slicemap.map.selected].values)
            current_plot_data = sub_sampled_data[slicemap.map.selected]
    if selected_tab == 0:
        #this is the load tab
        pass
    if selected_tab == 1:
        #this is the time slice tab
        if (len(slicemap.map.selected) != 0) and (True not in pd.isnull(selected_parameters_plot_data)):
            #aqui, ao inves de escrever direto para a linha, chamar o metodo de update com sub_sampled_data para x e selected para y
            slice_plot.update_plot(sub_sampled_data.index.values, selected_parameters_plot_data)
            detail_plot.update_plot(sub_sampled_data.index.values, selected_parameters_plot_data)
            slice_plot.fig.title = str(slicemap.map.selected)
            detail_plot.fig.title = str(slicemap.map.selected) + ' - Zoomed'
    elif selected_tab == 2:
        #this is the save to disk tab
        pass
    elif selected_tab == 3:
        #this is the analysis tab
        if len(slicemap.map.selected) != 0:
            analysis_plot.update_plot_data(sub_sampled_data.index.values, selected_parameters_plot_data)
            analysis_plot.fig.title = str(slicemap.map.selected)
            if time_slices_db_radio.value:
                analysis_plot.x_data_slice_min = time_slices_db[time_slices_db_radio.value][1]
                analysis_plot.x_data_slice_max = time_slices_db[time_slices_db_radio.value][2]
                analysis_plot.update_plot(current_plot_data, slicemap.map.selected, time_slices_db[time_slices_db_radio.value][1],
                                          time_slices_db[time_slices_db_radio.value][2], poly_order)
            else:
                update_analysis_plot(current_plot_data, slicemap.map.selected, analysis_plot, poly_order, \
                                 analysis_plot_zoom_slider)
    elif selected_tab == 4:
        #strip chart tab
        pass
    elif selected_tab == 5:
        #filter tab
        pass
    else:
        pass

# link selection handler to parameter map
slicemap.map.observe(selected_index_changed, 'selected')

############################### Plot Button Logic #############################################################################
plot_button = widgets.Button(description='Plot Slice Below')

def on_plot_button_clicked(b):
    lib_on_plot_button_clicked(b, cpt=current_plot_data, detplt=detail_plot, slcbx=slicebox, dtstats=detail_plot_stats)
    
plot_button.on_click(on_plot_button_clicked)


############################## File Chooser Logic #############################################################################

# Change defaults and reset the dialog
f.default_path = '/home/jovyan/work/data/'
f.default_filename = 'output.csv'
f.reset()
f.show_hidden = False
# Filter for csv only (missing CSV in capital)
f.filter_pattern = '*.csv'
f.use_dir_icons = True
f.show_only_dirs = False
f.title = 'Select <b>csv</b> file:'

def change_title(chooser):
    global raw_data, filetype, file_status_label, sub_sampled_data, map_groups, sub_sample_rate
    global current_plot_data, time_slices_db
    chooser.title = ''
    slicemap.map.selected = []
    data_statistics_label1.value = ' '
    data_statistics_label2.value = '___patience is a virtue___ '
    data_statistics_label3.value = ' '
    raw_data, filetype = load_data(unit_test, f, file_status_label)
    sub_sample_rate, _ = get_sub_sample_rate(raw_data, desired_sub_sample_rate)

    file_status_label.value = 'data finished reading.'

    data_statistics_label1.value = f'Imported data from: {raw_data.index[0]} to: {raw_data.index[-1]}, with {len(raw_data.index)} records'
    data_statistics_label2.value = f'Original data at {counter-1} Hz. Data subsampled to {desired_sub_sample_rate} Hz to speed up graphing. Reduced to {len(sub_sampled_data.index)} records'
    data_statistics_label3.value = 'Done.'

    sub_sampled_data = raw_data[::sub_sample_rate]
    map_groups = {}
    for p in sub_sampled_data.columns:
        map_groups[p] = 'FTI'
    map_groups = get_param_group(chooser.selected, filetype, raw_data, map_groups)
    slicemap.update_map(sub_sampled_data, map_groups)
    
    current_plot_data = sub_sampled_data

    slice_plot.update_plot(current_plot_data.index.values, current_plot_data.index.values)
    detail_plot.update_plot(current_plot_data.index.values, current_plot_data.index.values)
    slicebox.update_values(current_plot_data.index[0].to_datetime64(), current_plot_data.index[-1].to_datetime64())
    time_slices_db = {}
    time_slices_db_radio.options = []
    strip_chart.__init__(raw_data, slicemap, time_slices_db, time_slices_db_radio)


# Register callback function to filechooser
f.register_callback(change_title)

########################## Save Slice Button and Logic #############################################################################

# time slices are saved to a dictionary for later analysis
TP_number_box = widgets.FloatText(value=0.0, description='TP#:', disabled=False, layout=Layout(width='130px'))

TP_desc_box = widgets.Text(value='', placeholder='Type something', description='TP Title:', disabled=False)
TS_message = Label()


################## Data Slice Selector Instance #############################################################################
data_slice_selec = DataSliceSelect(time_slices_db)

################## Add/Remove Time Slices to db Logic #############################################################################

def update_TP_saved_display(time_slices_db):
    display_text = 'TP#  Description \n'
    for key in time_slices_db:
        display_text = display_text  + str(key) + '  ' + time_slices_db[key][0] + '\n'
    return display_text

        
def on_saveTS_button_clicked(b):
    lib_on_saveTS_button_clicked(b, cpt=current_plot_data, detplt=detail_plot, slcplt=slice_plot, \
                                tsdb=time_slices_db, tpnb=TP_number_box, tsmsg=TS_message, \
                                tpdescb=TP_desc_box, tpsavdd=TP_saved_dd, tsdbrb=time_slices_db_radio)
        

def on_delTS_button_clicked(b):
    lib_on_delTS_button_clicked(b, tsdb=time_slices_db, tsmsg=TS_message, \
                                tpsavdd=TP_saved_dd, tsdbrb=time_slices_db_radio)

TP_saved_dd = widgets.Dropdown(options=time_slices_db, description='TP#:', disabled=False,)

    
saveTS_button = widgets.Button(description='Save TSlice')
saveTS_button.on_click(on_saveTS_button_clicked)

delTS_button = widgets.Button(description='Delete TSlice')
delTS_button.on_click(on_delTS_button_clicked)

################## TS Radio Buttons #############################################################################

time_slices_db_radio = widgets.RadioButtons(options=[], description='Slices:', disabled=False)

def on_click_time_slices_db_radio(change):
    lib_on_click_time_slices_db_radio(change, cpt=current_plot_data, anplt=analysis_plot, \
                                     tsdb=time_slices_db, pord=poly_order, \
                                     slcmap=slicemap, zsld=analysis_plot_zoom_slider)

time_slices_db_radio.observe(on_click_time_slices_db_radio, 'value')

################## Save Slices to Disk Logic #############################################################################
        
def on_save_slices_button_clicked(b):
    lib_save_slices(tsdb=time_slices_db, svall=save_all_parameters, rawdt=raw_data, \
                   slcmap=slicemap, svfb=save_feedback)     

save_instructions = widgets.Label(value='Select parameters to save or select checkbox below to save all', disable=False)

save_feedback = widgets.Label(value='', disable=False)

save_all_parameters = widgets.Checkbox(value=False, description='save ALL PARAMETERS', disabled=False)

save_slices_button = widgets.Button(description='Save Slices to Disk')
save_slices_button.on_click(on_save_slices_button_clicked)
## we need to create the download button logic here
# create_download_link(sub_sampled_data)
# need to display the button, etc



################# Analysis Plot Definition #############################################################################

analysis_plot = AnalysisPlot(current_plot_data.index, current_plot_data.index.values)
poly_order = 1

analysis_poly_order_dd = widgets.Dropdown(options=[1,2,3,4], description='Poly Deg', value=1, disabled=False)


def on_change_analysis_poly_order_dd(change):
    global poly_order, current_plot_data
    if change['type'] == 'change' and change['name'] == 'value' and len(slicemap.map.selected) != 0:
        poly_order = change['new']
        update_analysis_plot(current_plot_data, slicemap.map.selected, analysis_plot, poly_order, analysis_plot_zoom_slider)
            
analysis_poly_order_dd.observe(on_change_analysis_poly_order_dd)

analysis_plot_zoom_slider = SimpleZoomSlider(analysis_plot)

def on_analysis_zoom_slider_change(change):
    update_analysis_plot(current_plot_data, slicemap.map.selected, analysis_plot, poly_order, analysis_plot_zoom_slider)


analysis_plot_zoom_slider.LH_slider.observe(on_analysis_zoom_slider_change, names='value')
analysis_plot_zoom_slider.RH_slider.observe(on_analysis_zoom_slider_change, names='value')


def on_slice_trim_button_clicked(b):
    slice_trim(zsld=analysis_plot_zoom_slider, anplt=analysis_plot, tsdb=time_slices_db, tsdbr=time_slices_db_radio, \
              cpt=current_plot_data, pord=poly_order, slcmap=slicemap)

    
slice_trim_button = widgets.Button(description='Trim Current Slice')
slice_trim_button.on_click(on_slice_trim_button_clicked)

################# Strip Charts Definition #############################################################################

strip_chart_button = widgets.Button(description='Plot Strip Chart Below')

strip_chart = StripChart(raw_data, slicemap, time_slices_db, time_slices_db_radio)

################ Filter Definitions #############################################################################

### bandpass

filter_label = widgets.Label('Low Pass Filter Parameters:')

filter_order = widgets.BoundedIntText(
            value=2,
            min=1,
            max=8,
            step=1,
            description='Filter Order:',
            disabled=False,
            layout=Layout(width='140px')
        )

filter_Wn = widgets.BoundedFloatText(
            value=0.01,
            min=0.001,
            max=0.999,
            step=0.01,
            description='3db kink pt:',
            disabled=False,
            layout=Layout(width='140px')
        )

apply_filter_button = widgets.Button(description='Apply Filter')

    
def on_apply_filter_button_clicked(b):
    global sub_sampled_data
    filter_data = raw_data[slicemap.map.selected].columns
    N = filter_order.value #filter order
    Wn = filter_Wn.value #3db cutoff kink point
    B, A = signal.butter(N, Wn, output='ba')
    
    for i in filter_data:
        if '_filt' not in i:
            raw_data[i+'_filt'] = signal.filtfilt(B, A, raw_data[i])
    
    for p in raw_data.columns:
        if not map_groups.get(p):
            map_groups[p] = 'filter'

    sub_sampled_data = raw_data[::sub_sample_rate]
    slicemap.update_map(sub_sampled_data, map_groups)


apply_filter_button.on_click(on_apply_filter_button_clicked)

### positive peak shaver

PS_pos_filter_label = widgets.Label('Positive Peak Shaver:')

PS_pos_filter_threshold = widgets.BoundedFloatText(
            value=0,
            min=-9999999999,
            max=9999999999,
            step=0.01,
            description='+Filter threshold:',
            disabled=False,
            layout=Layout(width='140px')
        )

apply_pos_PS_filter_button = widgets.Button(description='Apply Pos Peak Shaver Filter')

    
def apply_pos_PS_filter_button_clicked(b):
    global sub_sampled_data
    filter_data = raw_data[slicemap.map.selected].columns
    threshold = PS_pos_filter_threshold.value #filter threshold

    
    for i in filter_data:
        if '_PSPfilt' not in i:
            raw_data[i+'_PSPfilt'] = raw_data[i]
            raw_data.loc[raw_data[i+'_PSPfilt'].values > threshold, i+'_PSPfilt'] = np.nan
            raw_data.fillna(method='ffill', inplace=True)
            raw_data.fillna(method='bfill', inplace=True)
    
    for p in raw_data.columns:
        if not map_groups.get(p):
            map_groups[p] = 'filter'

    sub_sampled_data = raw_data[::sub_sample_rate]
    slicemap.update_map(sub_sampled_data, map_groups)


apply_pos_PS_filter_button.on_click(apply_pos_PS_filter_button_clicked)

### negative peak shaver
PS_neg_filter_label = widgets.Label('Negative Peak Shaver:')

PS_neg_filter_threshold = widgets.BoundedFloatText(
            value=0,
            min=-9999999999,
            max=9999999999,
            step=0.01,
            description='-Filter threshold:',
            disabled=False,
            layout=Layout(width='140px')
        )

apply_neg_PS_filter_button = widgets.Button(description='Apply Neg Peak Shaver Filter')

    
def apply_neg_PS_filter_button_clicked(b):
    global sub_sampled_data
    filter_data = raw_data[slicemap.map.selected].columns
    threshold = PS_neg_filter_threshold.value #filter threshold

    
    for i in filter_data:
        if '_PSNfilt' not in i:
            raw_data[i+'_PSNfilt'] = raw_data[i]
            raw_data.loc[raw_data[i+'_PSNfilt'].values < threshold, i+'_PSNfilt'] = np.nan
            raw_data.fillna(method='ffill', inplace=True)
            raw_data.fillna(method='bfill', inplace=True)
    
    for p in raw_data.columns:
        if not map_groups.get(p):
            map_groups[p] = 'filter'

    sub_sampled_data = raw_data[::sub_sample_rate]
    slicemap.update_map(sub_sampled_data, map_groups)


apply_neg_PS_filter_button.on_click(apply_neg_PS_filter_button_clicked)

#####################################################################################################
# MAIN TABS DEFINITIONS
#####################################################################################################

tab_contents = ['Load', 'Slice', 'Save', 'Analysis', 'StripChart', 'Filter']
load_tab_children = VBox([file_status_label, f, data_statistics_label1, data_statistics_label2, data_statistics_label3])
text_children = [widgets.Text(description=name) for name in tab_contents]

slice_tab_children = VBox([slice_plot.fig, slicebox.slice_box, plot_button, detail_plot.fig, \
                           VBox([TS_message, HBox([saveTS_button, TP_number_box, TP_desc_box, TP_saved_dd, delTS_button])])]) 

save_tab_children = VBox([save_instructions, save_all_parameters, save_slices_button, save_feedback])

anplttit = widgets.Label('______________________________________________________________________________________________________________________________________________________________')
plot_tab_children = VBox([anplttit, analysis_poly_order_dd,  analysis_plot.fig,  analysis_plot.fit_statistics, \
                          analysis_plot_zoom_slider.LH_slider, analysis_plot_zoom_slider.RH_slider, slice_trim_button])

strip_chart_children = strip_chart.widget()

filter_children = VBox([VBox([filter_label, filter_order, filter_Wn,apply_filter_button]), 
                        VBox([PS_pos_filter_label, PS_pos_filter_threshold, apply_pos_PS_filter_button]),
                        VBox([PS_neg_filter_label, PS_neg_filter_threshold, apply_neg_PS_filter_button])])

complete_tabs_children = [load_tab_children, slice_tab_children, save_tab_children, plot_tab_children, strip_chart_children, filter_children]
FTAT_tabs = widgets.Tab()
FTAT_tabs.children = complete_tabs_children

for i in range(len(complete_tabs_children)):
    FTAT_tabs.set_title(i, tab_contents[i])
    
FTAT_tabs.layout = Layout(width='100%')

# callbacks

def update_logic(param01):
    selected_tab = FTAT_tabs.selected_index
    #reset the saved files feedback everytime the tabs as changed
    save_feedback.value = ''
    if selected_tab == 0:
        selected_index_changed(None)
    if selected_tab == 1:
        selected_index_changed(None)
    elif selected_tab == 2:
        pass
    elif selected_tab == 3:
        selected_index_changed(None)
    elif selected_tab == 4:
        pass
    else :
        pass

def on_FTAT_tabs_change(change):
    #this is just a wrapper function that will  me to pass other parameters/objects
    #for now, just call the updater
    update_logic(1)
    
FTAT_tabs.observe(on_FTAT_tabs_change)

HBox([VBox(children = [slicemap.map, time_slices_db_radio], layout=Layout(width='40%')), 
      Box(children = [FTAT_tabs])])